In [2]:
from FiinQuantX import FiinSession


username = 'DSTC_20@fiinquant.vn'
password = 'Fiinquant0606'

client = FiinSession(
    username=username,
    password=password,
).login()

fi = client.FiinIndicator()    

In [3]:
import pandas as pd
df = pd.read_csv('ratios_non_null.csv')

In [4]:
indicator_cols = [
    'DebtToEquityRatio', 'EBITMargin', 'ROA', 'ROE', 'BasicEPS', 
    'PriceToBook', 'PriceToEarning', 'ROIC', 'NetRevenueGrowthYoY', 
    'GrossProfitGrowthYoY'
]

# Chuyển đổi các cột chỉ số sang dạng số, các giá trị không hợp lệ sẽ thành NaN
for col in indicator_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("⚙️ Đang tính toán giá trị trung bình cho mỗi mã...")

# Nhóm theo 'ticker' và tính giá trị trung bình cho các cột chỉ số
# Đồng thời lấy năm (year) và quý (quarter) gần nhất để tham khảo
df_avg = df.groupby('ticker').agg({
    'year': 'last',
    'quarter': 'last',
    'DebtToEquityRatio': 'mean',
    'EBITMargin': 'mean',
    'ROA': 'mean',
    'ROE': 'mean',
    'BasicEPS': 'mean',
    'PriceToBook': 'mean',
    'PriceToEarning': 'mean',
    'ROIC': 'mean',
    'NetRevenueGrowthYoY': 'mean',
    'GrossProfitGrowthYoY': 'mean'
}).reset_index()

# Đổi tên các cột trung bình để dễ nhận biết
avg_cols_rename = {col: f"{col}_Avg" for col in indicator_cols}
df_avg = df_avg.rename(columns=avg_cols_rename)

print("✅ Hoàn thành!")

# Lưu kết quả ra file mới
df_avg.to_csv('FA_averaged.csv', index=False)
print("\n💾 Kết quả đã được lưu vào file: FA_averaged.csv")

# Hiển thị 10 dòng đầu tiên của kết quả
print("\n📊 Dữ liệu sau khi tính trung bình:")
display(df_avg.head(10))

⚙️ Đang tính toán giá trị trung bình cho mỗi mã...
✅ Hoàn thành!

💾 Kết quả đã được lưu vào file: FA_averaged.csv

📊 Dữ liệu sau khi tính trung bình:


,ticker,year,quarter,DebtToEquityRatio_Avg,EBITMargin_Avg,ROA_Avg,ROE_Avg,BasicEPS_Avg,PriceToBook_Avg,PriceToEarning_Avg,ROIC_Avg,NetRevenueGrowthYoY_Avg,GrossProfitGrowthYoY_Avg
0,A32,2024,5,NaN,0.041139,0.073963,0.173339,5664.952870,1.129964,6.438798,0.165597,0.039047,0.022640
1,AAA,2025,2,0.531079,0.025163,0.022573,0.050560,679.867810,0.622881,14.537305,0.035063,0.026359,0.103977
2,AAH,2025,2,0.060158,0.078386,0.028351,0.032940,136.360372,0.394367,32.713080,0.034744,1.248406,1.044041
3,AAM,2025,2,NaN,-0.008558,0.013040,0.013542,188.254025,0.446416,-40.826328,-0.015680,0.215376,4.006879
4,AAS,2025,2,0.562141,0.433700,0.055614,0.083046,1242.344920,1.288499,41.640530,0.101354,-0.192746,-0.011467
5,AAT,2025,2,0.414393,0.058002,0.014421,0.020585,310.113633,0.385466,-427.174169,0.086358,0.117178,0.253563
6,AAV,2025,2,0.151812,-0.174081,-0.011144,-0.014337,-153.679394,0.533165,21.820579,-0.012697,5.242417,-0.208664
7,ABA,2024,5,1.818498,0.007576,-0.169014,-1.855981,NaN,NaN,NaN,0.011934,0.083582,-0.072809
8,ABC,2025,2,0.291986,0.127099,0.247733,1.720843,10111.367168,0.493874,14.107978,0.763704,0.081635,0.939676
9,ABI,2025,2,NaN,0.060144,0.056461,0.149897,2872.726249,1.260102,10.538617,0.063903,0.070139,-0.012008


In [5]:
# Áp dụng bộ lọc
# Tạo các điều kiện lọc
condition1 = df_avg['ROE_Avg'] > 0
condition2 = df_avg['DebtToEquityRatio_Avg'] < 2.5
condition3 = df_avg['NetRevenueGrowthYoY_Avg'] > 0

# Áp dụng bộ lọc
bare_minimum_df = df_avg[condition1 & condition2 & condition3].copy()

# Xử lý các giá trị NaN trong cột D/E (nếu có)
# Một số công ty không có nợ, D/E sẽ là NaN. Chúng ta muốn giữ lại các công ty này.
no_debt_companies = df_avg[df_avg['DebtToEquityRatio_Avg'].isnull()]
bare_minimum_df = pd.concat([bare_minimum_df, no_debt_companies[condition1 & condition3]], ignore_index=True).drop_duplicates(subset=['ticker'])


print(f"\n✅ HOÀN THÀNH: Có {len(bare_minimum_df)} mã vượt qua bộ lọc tối thiểu.")

# Lưu kết quả cuối cùng
bare_minimum_df.to_csv('FA_bare_minimum_selection.csv', index=False)
print("\n💾 Kết quả đã được lưu vào file: FA_bare_minimum_selection.csv")

# Hiển thị kết quả
print("\n🏆 Danh sách các mã cổ phiếu vượt qua bộ lọc tối thiểu:")
display(bare_minimum_df.sort_values('ROE_Avg', ascending=False))


✅ HOÀN THÀNH: Có 932 mã vượt qua bộ lọc tối thiểu.

💾 Kết quả đã được lưu vào file: FA_bare_minimum_selection.csv

🏆 Danh sách các mã cổ phiếu vượt qua bộ lọc tối thiểu:


,ticker,year,quarter,DebtToEquityRatio_Avg,EBITMargin_Avg,ROA_Avg,ROE_Avg,BasicEPS_Avg,PriceToBook_Avg,PriceToEarning_Avg,ROIC_Avg,NetRevenueGrowthYoY_Avg,GrossProfitGrowthYoY_Avg
35,ATG,2025,2,-45.749465,-0.792775,-0.439227,20.263053,-2032.715437,284.035225,-10.806209,-0.122749,2.848088,13.447367
674,TSD,2024,5,1.498126,0.059244,0.136920,13.573429,1263.966004,16.636754,4.120213,-4.545609,0.460991,0.751937
507,PPI,2022,5,-0.441871,13.945507,-0.234419,6.057935,-1436.659503,-0.513311,-2.615685,-0.377485,5.714367,-1.742218
187,DIC,2025,2,-11.452252,-0.211918,-0.018407,5.145754,-802.370291,-0.453714,10.591999,0.061131,14.805316,3.913439
374,LQN,2024,5,-2.344465,-0.268548,-0.058266,3.612398,-3791.666630,-0.404482,-1.256211,0.421995,0.626814,24.865772
...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,UNI,2025,2,1.358194,-0.776572,0.000419,0.001133,11.907872,0.865515,31054.343286,-0.000403,1.016055,1.096151
539,PXL,2025,2,0.013094,-1.350901,0.000740,0.000783,7.986533,1.412732,1855.330029,-0.007009,0.173540,0.455730
536,PVX,2025,2,1.315587,-0.047935,-0.001342,0.000405,-23.855372,3.552332,28.976965,-0.052945,0.068475,2.130358
607,SSN,2021,5,0.004291,-0.234430,0.000044,0.000109,1.243208,0.631229,7245.546684,-0.001613,0.425058,0.068043
